In [ ]:
!pip install ../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/ > /dev/null # no output
!pip install torch_optimizer --no-index --find-links=file:///kaggle/input/torch-optimizer/torch_optimizer
    
import math
import pandas as pd
import os 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer as optim
from torch.optim import lr_scheduler
import torchvision.models as models
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import sklearn.model_selection
import torchvision.datasets
import torchvision.transforms as transforms
import torch
import torchvision
from PIL import Image
from sklearn.metrics import mean_squared_error
import efficientnet_pytorch
from glob import glob
import gc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_data = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
# target_data = train_data['Pawpularity']
train_data

In [ ]:
train_images_path = []
for dirpath, dirnames, filenames in os.walk("../input/petfinder-pawpularity-score/train"):
    for filename in [f for f in filenames if f.endswith(".jpg")]:
        train_images_path.append(os.path.join(dirpath, filename))
train_images_path
train_data['input_file_loc'] = sorted(train_images_path)
train_data

In [ ]:
train_data.iloc[0,14]

In [ ]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
features = train_data['Pawpularity'].values
features

In [ ]:
class PetDataset(torch.utils.data.Dataset):
    def __init__(self, df, data_dir, transform=None, mode='train'):
        self.df = df
        self.data_dir = data_dir
        self.transform = transform
        self.mode = mode
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.df.iloc[idx, 14]
        dense_features = self.df.iloc[idx, 1:13]
        img = Image.open(img_name)
        
        if self.transform is not None:
            img = self.transform(img)
        target = torch.tensor([self.df.iloc[idx, 13]])
        dense_features = torch.tensor(dense_features)
        return img, dense_features, target

In [ ]:
class PawpularModel(nn.Module):
    def __init__(self, depth):
        super(PawpularModel, self).__init__()
        self.depth = depth
        self.base = efficientnet_pytorch.EfficientNet.from_name(f'efficientnet-b{self.depth}' )
        pretrained_file = glob(f'../input/efficientnet-pytorch/efficientnet-b{self.depth}*')[0]
        checkpoint = torch.load(pretrained_file)
        self.base.load_state_dict(checkpoint)
#         self.model = timm.create_model(model_name, pretrained=False, in_chans=3)
        self.base._fc = nn.Linear(self.base._fc.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(140, 64)
        self.fc2 = nn.Linear(64, 1)
        self.prelu = nn.PReLU()

    def forward(self, image, features):
        x = self.base(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = self.prelu(x)
        return x

In [ ]:
def radam(parameters, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
    if isinstance(betas, str):
        betas = eval(betas)
    return optim.RAdam(parameters,
                      lr=lr,
                      betas=betas,
                      eps=eps,
                      weight_decay=weight_decay)

In [ ]:
criterion = nn.MSELoss()

In [ ]:
from torch.autograd import Variable
#add extra one columns
# train_data['kfold']=-1
#Distributing the data 5 shares
kfold = KFold(n_splits=10, shuffle= True, random_state = 0)
for fold, (train_indicies, valid_indicies) in enumerate(kfold.split(X=train_data)):
    model = PawpularModel(0)
    lr = None
    optimizer = radam(model.parameters(), lr=1e-3, betas=(0.9,0.999), eps=1e-3, weight_decay=1e-4)
    model.to(device)
#     model.train()
    train_x, valid_x = train_data.loc[train_indicies], train_data.loc[valid_indicies]
    dataset  = PetDataset(df=train_x, data_dir="../input/petfinder-pawpularity-score/train", transform=transform, mode='train')
    valid_dataset  = PetDataset(df=valid_x, data_dir="../input/petfinder-pawpularity-score/train", transform=transform, mode='vaild')
    train_loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=64, shuffle=True, num_workers=0)
    vaild_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=64, shuffle=True, num_workers=0)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*10, eta_min=1e-6)
    for epoch in range(10):
        running_loss = 0.0
        error = 0.0
        vaild_num = 0
        model.train()
        for i, data in enumerate(train_loader, 0):
            inputs, features, labels = data
#             inputs, features, labels = Variable(inputs.to(device)), Variable(features.to(device)), Variable(labels.to(device))
#             print(inputs.shape,features.shape,labels.shape)
            
            features=Variable(features.cuda().to(torch.float32))
            inputs=Variable(inputs.cuda().to(torch.float32))
            labels=Variable(labels.cuda().to(torch.float32))
            
            outputs = model(inputs, features)
            outputs=outputs.to(torch.float32)
        
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            lr = optimizer.param_groups[0]['lr']
            running_loss += loss.item()
            if i % 20 == 19:    # print every 20 mini-batches
                print('[%d, %5d] loss: %.3f RMSE: %.3f'%
                      (epoch + 1, i + 1, running_loss / 20, mean_squared_error(labels.cpu().detach().numpy(), outputs.detach().cpu().numpy(), squared=False)))
                running_loss = 0.0
        model.eval()
        for i, vaild in enumerate(vaild_loader, 0): 
            inputs, features, labels = vaild
            inputs, features, labels = inputs.to(device), features.to(device), labels.to(device)
            outputs = model(inputs, features)
            labels=labels.to(torch.float32)
            outputs=outputs.to(torch.float32)
            error += mean_squared_error(labels.cpu().detach().numpy(), outputs.detach().cpu().numpy(), squared=False)
            vaild_num = i
        print('error = ', error/(vaild_num+1))
    
    torch.save(model.state_dict(), f'./fold_{fold}.pth')
    del model
    gc.collect()
    torch.cuda.empty_cache()
#     dataiter = iter(train_loader)
#     images, features, labels = dataiter.next()

    
# print(train_data.kfold.value_counts()) #total data 300000 = kfold split :5 * 60000

#output of train folds data
# train_data.to_csv("trainfold_10.csv",index=False)
# train_data